# Introdução
## Contexto do projeto
Todos os anos letivos, a equipa administrativa do **IPCA** enfrenta dificuldades na criação dos horários das aulas, devido às restrições complexas relacionadas com **professores, cursos, horários e salas disponíveis**. Este é um problema típico de **Satisfação de Restrições (*Constraint Satisfaction Problem* - CSP)**, onde é necessário encontrar soluções que satisfaçam um conjunto de *hard constraints* e *soft constraints*.

## Objetivo do projeto
O principal objetivo do projeto é criar um sistema que produza horários **válidos, otimizados e que respeitem todas as restrições**.

## Equipa de desenvolvimento
- Duarte Pereira — Nº 27959
- Hugo Especial — Nº 27963
- Paulo Gonçalves — Nº 27966 
- Marco Cardoso — Nº 27969 
- Hugo Pereira — Nº 27970

---

# *Design* do agente

## Formulação do problema como um Problema de Satisfação de Restrições (CSP)
Formular um problema de CSP exige identificar claramente três elementos:
- Variáveis, domínios e restrições.

E o objetivo é encontrar uma atribuição de valores que satisfaça todas as restrições.

## Definição de variáveis, domínio e restrições
### **Variáveis**:
$X = \{ Sala, Dia, Hora \}$

### **Domínio**:  
- $D(Sala) = \{ (Lab01, Lab02, Lab03) \}$
- $D(Dia) = \{ (Segunda, Terça, Quarta, Quinta, Sexta) \}$
- $D(Hora) = \{ (9h00 - 11h00, 11h00 - 13h00, 14h00 - 16h00, 16h00 - 18h00) \}$

### **Restrições**:
#### ***Hard***

- Cada aula dura **2 horas**.
Esta função não está explicíta no código.
```python
def check_duration(aula): 
```

- As turmas **não podem ter a mesma sala durante a mesma hora**. 
```python
def no_same_room_same_time(a1, a2):
```

- As turmas **não podem ter duas aulas no mesmo bloco horário.**. 
```python
def no_same_turma_same_time(a1, a2):
```

- Um professor **não podem ter duas aulas no mesmo horário**. 
```python
def no_same_professor_same_time(a1, a2):
```

- As turmas **não podem ter mais de 3 aulas por dia**. 
```python
def max_three_per_day_turma(*aulas):
```

- Um aluno **não pode ter duas aulas da mesma UC no mesmo dia**. 
```python
def same_uc_different_days(a1, a2):
```

- Um aluno **têm que ter 2 aulas da cada UC por semana**. 
```python
def exactly_two_per_uc(*aulas, ucs=None):
```

- As turmas **têm de ter 10 aulas por semana**. 
```python
def exactly_ten_per_turma(*aulas, turmas=None):
```

### Importação dos dados
Os dados foram importados a partir de ficheiros de texto (.txt), utilizando Python. O ficheiro contém informações sobre diferentes entidades (cursos, professores, salas, turmas, unidades curriculares e disponibilidades.)

In [ ]:
def load_data_txt(path='dados/data.txt'):
    data = {
        'classes': {},           # #cc — cursos atribuídos a turmas
        'one_lesson_week': [],   # #olw — cursos com apenas uma aula/semana
        'teachers': {},          # #dsd — professores e UCs atribuídas
        'time_restrictions': {}, # #tr — restrições de horários (NÃO disponíveis)
        'room_restrictions': {}, # #rr — restrições de salas
        'online_classes': {},    # #oc — aulas online
        'time_availabilities': {} # calculadas automaticamente (disponíveis)
    }

    current_section = None

    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('—') or line.startswith('#head'):
                continue

            # Identificar secções
            if line.startswith('#'):
                if line.startswith('#cc'):
                    current_section = 'classes'
                elif line.startswith('#olw'):
                    current_section = 'one_lesson_week'
                elif line.startswith('#dsd'):
                    current_section = 'teachers'
                elif line.startswith('#tr'):
                    current_section = 'time_restrictions'
                elif line.startswith('#rr'):
                    current_section = 'room_restrictions'
                elif line.startswith('#oc'):
                    current_section = 'online_classes'
                else:
                    current_section = None
                continue

            parts = line.split()
            if not parts:
                continue

            if current_section == 'classes':
                turma = parts[0]
                ucs = parts[1:]
                data['classes'][turma] = ucs

            elif current_section == 'one_lesson_week':
                data['one_lesson_week'].append(parts[0])

            elif current_section == 'teachers':
                professor = parts[0]
                ucs = parts[1:]
                data['teachers'][professor] = ucs

            elif current_section == 'time_restrictions':
                professor = parts[0]
                not_available = [int(x) for x in parts[1:]]
                data['time_restrictions'][professor] = not_available

            elif current_section == 'room_restrictions':
                uc, sala = parts
                data['room_restrictions'][uc] = sala

            elif current_section == 'online_classes':
                uc, idx = parts
                data['online_classes'][uc] = int(idx)

    # Converter restrições em disponibilidades para todos os professores
    all_slots = set(range(1, 21))  # 20 blocos (5 dias × 4 blocos)
    for prof in data['teachers'].keys():  # garantir todos os professores
        unavailable = data['time_restrictions'].get(prof, [])  # se não tiver, assume []
        available = sorted(all_slots - set(unavailable))
        data['time_availabilities'][prof] = available

    return data


if __name__ == "__main__":
    dados = load_data_txt()

    print("=== Professores e UCs ===")
    for prof, ucs in dados['teachers'].items():
        print(f"{prof}: {ucs}")

    print("\n=== Slots indisponíveis ===")
    for prof, slots in dados['time_restrictions'].items():
        print(f"{prof}: {slots}")

    print("\n=== Slots disponíveis ===")
    for prof, slots in dados['time_availabilities'].items():
        print(f"{prof}: {slots}")


## Função heurística para implementar as *soft constraints*


O processo de geração de horários é dividido em duas fases.
Na primeira, **o *solver* da biblioteca *Constraint* gera todas as soluções válidas que respeitam as *hard constraints*, garantindo apenas a validade das restrições obrigatórias**.

Na segunda, **as soluções são avaliadas e ordenadas através de uma função heurística**, que calcula uma **pontuação com base nas *soft constraints*,** refletindo o grau de otimização das preferências.

A função heurística calcula uma pontuação para cada solução, considerando critérios como:
- o número de dias distintos que uma UC ocupa.
- se uma turma tiver 4 ou menos dias de aulas.
- se as aulas de um dia forem consecutivas.
- o menor número de salas diferentes.

A pontuação total da solução é a **soma dos valores obtidos em cada uma destas preferências**.

As **soluções com maior pontuação são consideradas ótimas**, uma vez que cumprem melhor as preferências definidas.

---


In [ ]:
# Cada UC não deve ter aulas em mais de 2 dias diferentes
def check_distinct_day_classes(*aulas):
    uc_days = {}
    for aula in aulas:
        dia, _, _, _, _, uc = aula
        uc_days.setdefault(uc, set()).add(dia)
    return sum(len(dias) - 2 for dias in uc_days.values() if len(dias) > 2) == 0


# Cada turma não deve ter aulas em mais de 4 dias por semana
def check_weekly_days(*aulas):
    turma_days = {}
    for aula in aulas:
        dia, _, _, _, turma, _ = aula
        turma_days.setdefault(turma, set()).add(dia)
    return all(len(dias) <= 4 for dias in turma_days.values())


# Aulas de uma mesma turma num mesmo dia devem ser consecutivas (cada aula = 2h)
def check_consecutive_classes(*aulas):
    turma_dia_horas = {}
    for aula in aulas:
        dia, hora, _, _, turma, _ = aula
        turma_dia_horas.setdefault((turma, dia), []).append(hora)

    for horas in turma_dia_horas.values():
        horas_ordenadas = sorted(horas)
        for i in range(1, len(horas_ordenadas)):
            if horas_ordenadas[i] - horas_ordenadas[i-1] != 2:
                return False
    return True


# Cada turma deve usar no máximo 3 salas diferentes
def check_different_classes(*aulas):
    turma_salas = {}
    for aula in aulas:
        _, _, sala, _, turma, _ = aula
        turma_salas.setdefault(turma, set()).add(sala)
    return all(len(salas) <= 3 for salas in turma_salas.values())


## Análise Crítica

O agente consegue gerar horários válidos que respeitam todas as restrições obrigatórias e otimiza preferências definidas.

### Pontos positivos:
- Soluções válidas respeitam todas as *hard constraints*.
- A heurística permite identificar a solução com melhor pontuação, considerando:
  - Dias distintos por UC
  - Máximo de 4 dias de aulas por turma
  - Aulas consecutivas
  - Minimização de salas diferentes

### Limitações:
- O número de soluções cresce rapidamente com *datasets* maiores, tornando a execução mais lenta.
- A heurística não garante encontrar a solução globalmente ótima.
- Conflitos entre disponibilidade de professores e salas podem tornar algumas combinações inviáveis.

### Possível melhoria:
- Usar *backtracking* com heurísticas mais avançadas.
- Implementação de horário online.


## Conclusão

O projeto permitiu desenvolver um agente capaz de gerar horários de aulas que respeitam todas as restrições obrigatórias e otimizam preferências desejáveis.

Permitiu compreender a formulação de CSPs, implementação prática de *hard* e *soft constraints* e utilização do Jupyter Notebook como ferramenta de documentação.

O agente pode ser melhorado para lidar com *datasets* maiores ou restrições adicionais, mas já fornece uma solução funcional e automatizada para a geração de horários académicos no IPCA.
